In [2]:
import pandas as pd
import numpy as np # Used for pd.NA, though pandas handles it internally now
import os
import glob # For finding all CSV files in a directory

def clean_area_column_in_csv(input_csv_path, output_csv_path, column_name='面積（㎡）'):
    """
    Cleans the specified column in a CSV file, converting its values to integers.
    Non-numeric values will be converted to pandas.NA (which typically maps to NULL in BigQuery).

    Args:
        input_csv_path (str): The path to the input CSV file.
        output_csv_path (str): The path where the cleaned CSV file will be saved.
        column_name (str): The name of the column to clean (default is '面積___').
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input file not found at '{input_csv_path}'")
        return

    print(f"Reading CSV from: {input_csv_path}")
    try:
        # Read the CSV file. Using dtype=str for the target column to prevent pandas
        # from inferring types prematurely, giving us full control.
        # Ensure encoding is correct for your Japanese characters, 'utf-8' is common.
        df = pd.read_csv(input_csv_path, dtype={column_name: str}, encoding='utf-8')
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return

    # Check if the column actually exists in the DataFrame
    if column_name not in df.columns:
        print(f"Warning: Column '{column_name}' not found in '{input_csv_path}'. Skipping.")
        # Save the original file to the output path if the column isn't found,
        # or handle as per your requirement (e.g., skip saving).
        df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        return

    print(f"Cleaning column: '{column_name}' for file: {os.path.basename(input_csv_path)}")

    # Define a function to apply to each value in the column
    def to_integer_or_na(value):
        try:
            # Convert to string, strip whitespace, and handle common empty representations
            s_value = str(value).strip()
            if not s_value or s_value.lower() in ['nan', 'n/a', 'null', 'none']: # Add more if needed
                return pd.NA
            # Attempt to convert to float first to handle cases like "123.0"
            # Then convert to int, effectively truncating decimals
            return int(float(s_value))
        except (ValueError, TypeError):
            # If conversion fails, return pandas.NA.
            # pandas.NA is better for representing missing values and maps well to BigQuery NULL.
            return pd.NA

    # Apply the cleaning function to the specified column
    df[column_name] = df[column_name].apply(to_integer_or_na)

    # Convert the column to nullable integer type (Int64)
    # This ensures that pd.NA values are correctly handled as missing integers.
    df[column_name] = df[column_name].astype('Int64')

    print(f"Saving cleaned CSV to: {output_csv_path}")
    try:
        # Save the cleaned DataFrame to a new CSV file
        # index=False prevents pandas from writing the DataFrame index as a column
        # encoding='utf-8-sig' is often preferred for CSVs with non-ASCII chars, especially for Excel compatibility.
        df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        print(f"Successfully cleaned and saved: {os.path.basename(output_csv_path)}")
    except Exception as e:
        print(f"Error writing CSV file '{output_csv_path}': {e}")

# --- Main execution block ---
if __name__ == "__main__":
    # --- IMPORTANT: Before running this script ---
    # 1. Install pandas: pip install pandas
    # 2. Download your CSV files from GCS to a local folder.
    #    Example: gsutil cp -r gs://test123_eve/train/ ./original_csvs

    # Define your input and output folders
    # Replace './original_csvs' with the path where your original CSVs are stored locally
    input_folder = "/Users/evehuang/Downloads/archive/train/train"
    # Replace './cleaned_csvs' with the path where you want to save the cleaned CSVs
    output_folder = "/Users/evehuang/Downloads/clean"

    print(f"Starting CSV cleaning process.")
    print(f"Input folder: {os.path.abspath(input_folder)}")
    print(f"Output folder: {os.path.abspath(output_folder)}")

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Find all CSV files in the input folder
    csv_files = glob.glob(os.path.join(input_folder, "*.csv"))

    if not csv_files:
        print(f"No CSV files found in the input folder: {input_folder}")
    else:
        print(f"Found {len(csv_files)} CSV files to process.")
        # Loop through each CSV file and clean it
        for csv_file in csv_files:
            base_name = os.path.basename(csv_file) # Get just the filename (e.g., "01.csv")
            # Create a new name for the cleaned file (e.g., "cleaned_01.csv")
            output_path = os.path.join(output_folder, f"cleaned_{base_name}")

            clean_area_column_in_csv(csv_file, output_path)

    print("\nAll specified CSV files processed.")
    print(f"Cleaned files are located in: {os.path.abspath(output_folder)}")
    print("\n--- Next Steps ---")
    print("1. Upload the cleaned CSV files from the output folder to a GCS bucket.")
    print("   Example: gsutil cp -r ./cleaned_csvs/* gs://your-gcs-bucket/cleaned_train_data/")
    print("2. Load the cleaned files into BigQuery using the explicit schema:")
    print("   bq load \\")
    print("     --source_format=CSV \\")
    print("     --column_name_character_map=V2 \\")
    print("     --schema=./table_schema.json \\")
    print("     eveonegcp.japan_house_price \\")
    print("     gs://your-gcs-bucket/cleaned_train_data/*.csv")

Starting CSV cleaning process.
Input folder: /Users/evehuang/Downloads/archive/train/train
Output folder: /Users/evehuang/Downloads/clean
Found 47 CSV files to process.
Reading CSV from: /Users/evehuang/Downloads/archive/train/train/40.csv
Cleaning column: '面積（㎡）' for file: 40.csv
Saving cleaned CSV to: /Users/evehuang/Downloads/clean/cleaned_40.csv
Successfully cleaned and saved: cleaned_40.csv
Reading CSV from: /Users/evehuang/Downloads/archive/train/train/41.csv
Cleaning column: '面積（㎡）' for file: 41.csv
Saving cleaned CSV to: /Users/evehuang/Downloads/clean/cleaned_41.csv
Successfully cleaned and saved: cleaned_41.csv
Reading CSV from: /Users/evehuang/Downloads/archive/train/train/43.csv
Cleaning column: '面積（㎡）' for file: 43.csv
Saving cleaned CSV to: /Users/evehuang/Downloads/clean/cleaned_43.csv
Successfully cleaned and saved: cleaned_43.csv
Reading CSV from: /Users/evehuang/Downloads/archive/train/train/42.csv
Cleaning column: '面積（㎡）' for file: 42.csv
Saving cleaned CSV to: /User

In [3]:
import pandas as pd
import numpy as np
import os
import glob
import re # For regular expressions to parse strings

def convert_japanese_era_to_ad(era_year_str):
    """
    Converts Japanese era year strings (e.g., 昭和64年, 平成4年) to Gregorian (AD) year integers.
    Handles missing/invalid values by returning pd.NA.
    """
    if pd.isna(era_year_str) or not isinstance(era_year_str, str):
        return pd.NA

    s = era_year_str.strip()

    # Mapping for common Japanese eras to their start years
    era_map = {
        '明治': 1868,
        '大正': 1912,
        '昭和': 1926,
        '平成': 1989,
        '令和': 2019
    }

    for era_name, start_year in era_map.items():
        if s.startswith(era_name):
            try:
                # Extract the year number after the era name
                year_part = s[len(era_name):].replace('年', '').strip()
                if year_part == '元年': # 元年 means the first year
                    return start_year
                else:
                    era_relative_year = int(year_part)
                    return start_year + era_relative_year - 1
            except ValueError:
                return pd.NA # Handle cases where year part is not a valid number
    return pd.NA # If no matching era found or other issues

def convert_quarter_to_date(quarter_str):
    """
    Converts 'YYYY年第X四半期' strings to 'YYYY-MM-DD' date strings (first day of quarter).
    Handles missing/invalid values by returning pd.NA.
    """
    if pd.isna(quarter_str) or not isinstance(quarter_str, str):
        return pd.NA

    s = quarter_str.strip()
    match = re.match(r'(\d{4})年第(\d)四半期', s)
    if match:
        year = int(match.group(1))
        quarter = int(match.group(2))
        if 1 <= quarter <= 4:
            # Map quarter to month
            month = {1: 1, 2: 4, 3: 7, 4: 10}[quarter]
            return f"{year:04d}-{month:02d}-01"
    return pd.NA # If format doesn't match or quarter is invalid

def clean_csv_for_bq(input_csv_path, output_csv_path):
    """
    Cleans a CSV file for BigQuery upload by handling specific column types and missing values.

    Args:
        input_csv_path (str): The path to the input CSV file.
        output_csv_path (str): The path where the cleaned CSV file will be saved.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input file not found at '{input_csv_path}'")
        return

    print(f"Reading CSV from: {input_csv_path}")
    try:
        # Read the CSV file. Using dtype=str for potentially problematic columns
        # to prevent pandas from inferring types prematurely.
        # 'utf-8-sig' is good for CSVs with BOM, common for Excel/Japanese text.
        df = pd.read_csv(input_csv_path, encoding='utf-8-sig', dtype={
            '面積（㎡）': str,
            '最寄駅：距離（分）': str,
            '建築年': str,
            '取引時点': str
        })
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return

    print(f"Cleaning columns for file: {os.path.basename(input_csv_path)}")

    # --- Cleaning '面積（㎡）' (Area in sq meters) ---
    # Convert to integer, non-numeric/empty to pd.NA
    if '面積（㎡）' in df.columns:
        df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: int(float(str(x).strip())) if str(x).strip().replace('.', '', 1).isdigit() else pd.NA)
        df['面積（㎡）'] = df['面積（㎡）'].astype('Int64') # Use nullable integer type
    else:
        print("Warning: '面積（㎡）' column not found.")

    # --- Cleaning '最寄駅：距離（分）' (Distance to Nearest Station in minutes) ---
    # Convert '30分?60分' to midpoint (45) or pd.NA if other non-numeric
    if '最寄駅：距離（分）' in df.columns:
        def clean_station_distance(value):
            s_value = str(value).strip()
            if s_value == '30分?60分':
                return 45 # Midpoint of the range
            try:
                # Attempt to convert to float then int
                return int(float(s_value))
            except (ValueError, TypeError):
                return pd.NA
        df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(clean_station_distance)
        df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].astype('Int64') # Use nullable integer type
    else:
        print("Warning: '最寄駅：距離（分）' column not found.")

    # --- Cleaning '建築年' (Construction Year) ---
    # Convert Japanese era years to Gregorian years
    if '建築年' in df.columns:
        df['建築年'] = df['建築年'].apply(convert_japanese_era_to_ad)
        df['建築年'] = df['建築年'].astype('Int64') # Use nullable integer type
    else:
        print("Warning: '建築年' column not found.")

    # --- Cleaning '取引時点' (Transaction Date) ---
    # Convert 'YYYY年第X四半期' to 'YYYY-MM-DD'
    if '取引時点' in df.columns:
        df['取引時点'] = df['取引時点'].apply(convert_quarter_to_date)
        # BigQuery DATE type expects YYYY-MM-DD string. Keep as string after conversion.
    else:
        print("Warning: '取引時点' column not found.")

    # --- Handle other potential empty strings for numeric columns if they exist ---
    # For example, '建ぺい率（％）' and '容積率（％）' might have empty strings if they are numeric.
    # The current read_csv will likely infer them as float if mixed with numbers,
    # or object (string) if many empty. If they are intended as numeric,
    # and contain empty strings, ensure they become nullable floats.
    numeric_cols_to_check = ['建ぺい率（％）', '容積率（％）', '前面道路：幅員（ｍ）']
    for col in numeric_cols_to_check:
        if col in df.columns:
            # Convert empty strings to NaN, then to nullable float
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Float64')
        else:
            print(f"Warning: '{col}' column not found.")

    # --- Save the cleaned DataFrame to a new CSV file ---
    print(f"Saving cleaned CSV to: {output_csv_path}")
    try:
        # index=False prevents pandas from writing the DataFrame index as a column
        # encoding='utf-8-sig' is important for Japanese characters and Excel compatibility
        df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        print(f"Successfully cleaned and saved: {os.path.basename(output_csv_path)}")
    except Exception as e:
        print(f"Error writing CSV file '{output_csv_path}': {e}")

# --- Main execution block ---
if __name__ == "__main__":
    # --- IMPORTANT: Before running this script ---
    # 1. Install pandas: pip install pandas
    # 2. Download your CSV files from GCS to a local folder.
    #    Example: gsutil cp -r gs://test123_eve/train/ ./original_csvs

    # Define your input and output folders
    # Replace './original_csvs' with the path where your original CSVs are stored locally
    input_folder = "/Users/evehuang/Downloads/archive/train/train"
    # Replace './cleaned_csvs' with the path where you want to save the cleaned CSVs
    output_folder = "/Users/evehuang/Downloads/clean"

    print(f"Starting comprehensive CSV cleaning process.")
    print(f"Input folder: {os.path.abspath(input_folder)}")
    print(f"Output folder: {os.path.abspath(output_folder)}")

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Find all CSV files in the input folder
    csv_files = glob.glob(os.path.join(input_folder, "*.csv"))

    if not csv_files:
        print(f"No CSV files found in the input folder: {input_folder}")
    else:
        print(f"Found {len(csv_files)} CSV files to process.")
        # Loop through each CSV file and clean it
        for csv_file in csv_files:
            base_name = os.path.basename(csv_file) # Get just the filename (e.g., "01.csv")
            # Create a new name for the cleaned file (e.g., "cleaned_01.csv")
            output_path = os.path.join(output_folder, f"cleaned_{base_name}")

            clean_csv_for_bq(csv_file, output_path)

    print("\nAll specified CSV files processed.")
    print(f"Cleaned files are located in: {os.path.abspath(output_folder)}")
    print("\n--- Next Steps ---")
    print("1. Upload the cleaned CSV files from the output folder to a GCS bucket.")
    print("   Example: gsutil cp -r ./cleaned_csvs/* gs://your-gcs-bucket/cleaned_train_data/")
    print("2. Retrieve the updated schema for your table from BigQuery (important after cleaning).")
    print("   bq show --schema --format=prettyjson eveonegcp.japan_house_price > table_schema.json")
    print("   (Review table_schema.json to ensure types like '建築年' are INT64, '取引時点' is STRING, etc.)")
    print("3. Load the cleaned files into BigQuery using the explicit schema and skipping the header:")
    print("   bq load \\")
    print("     --source_format=CSV \\")
    print("     --column_name_character_map=V2 \\")
    print("     --schema=./table_schema.json \\")
    print("     --skip_leading_rows=1 \\")
    print("     eveonegcp.japan_house_price \\")
    print("     gs://your-gcs-bucket/cleaned_train_data/*.csv")


Starting comprehensive CSV cleaning process.
Input folder: /Users/evehuang/Downloads/archive/train/train
Output folder: /Users/evehuang/Downloads/clean
Found 47 CSV files to process.
Reading CSV from: /Users/evehuang/Downloads/archive/train/train/40.csv
Cleaning columns for file: 40.csv
Saving cleaned CSV to: /Users/evehuang/Downloads/clean/cleaned_40.csv
Successfully cleaned and saved: cleaned_40.csv
Reading CSV from: /Users/evehuang/Downloads/archive/train/train/41.csv
Cleaning columns for file: 41.csv
Saving cleaned CSV to: /Users/evehuang/Downloads/clean/cleaned_41.csv
Successfully cleaned and saved: cleaned_41.csv
Reading CSV from: /Users/evehuang/Downloads/archive/train/train/43.csv
Cleaning columns for file: 43.csv
Saving cleaned CSV to: /Users/evehuang/Downloads/clean/cleaned_43.csv
Successfully cleaned and saved: cleaned_43.csv
Reading CSV from: /Users/evehuang/Downloads/archive/train/train/42.csv
Cleaning columns for file: 42.csv
Saving cleaned CSV to: /Users/evehuang/Downloa